In [ ]:
#Operating via google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#Copying content created previously via pre processing (20%data)
!cp /content/gdrive/My\ Drive/ML\ Project/ProcessedData_20p_new.csv /content

In [ ]:
import pandas as pd

In [ ]:
col_Names=["Sequence", "TitleBody","Tags"]
data = pd.read_csv('/content/ProcessedData_20p_new.csv', encoding="utf-8",names=col_Names,header=None)
data = data[1:]

In [ ]:
#Considering only 0.5 of 20% of entire data as google colab hangs after 15%
data = data.sample(frac=0.0001) 
data.head()
data.shape

(60, 3)

In [ ]:

df = data
#df.head()
df.reset_index(drop=True, inplace=True)
#df.head()

In [ ]:
  #Import libraries for encoding 
  from sklearn.feature_extraction.text import CountVectorizer
  import numpy as np

In [ ]:
#Converting tags to binary one hot encoded tags
vector = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
binary_y = vector.fit_transform(df['Tags'])
print(binary_y.toarray().shape)

(60, 140)


In [ ]:
# we changed considered tags as per the dataset considered
# considered tags = top listed tags as per frequency 
ConsideredTags = 40
freq = binary_y.sum(axis=0).tolist()[0]
length = len(freq)

#sorting tags in descending order of occurance
sortedTags = sorted(range(length), key=lambda k: freq[k], reverse=True)
#Filtered binary tags excluding less occuring tags
filteredTags = binary_y[:,sortedTags[:ConsideredTags]]
#print(filteredTags.toarray())


In [ ]:
sumTags = filteredTags.sum(axis=1)
questions_with_no_tags =(np.count_nonzero(sumTags==0))
print("No of questions which won't be having tags:",questions_with_no_tags)
print("Percentage:",questions_with_no_tags*100/df.shape[0])

No of questions which won't be having tags: 18
Percentage: 30.0


In [ ]:
# Spliting data test:train = 20:80
x_train=df.head(int(0.80*df.shape[0]))
x_test=df.tail(df.shape[0] - int(0.80*df.shape[0]))

y_train = filteredTags[0:int(0.80*df.shape[0]),:]
y_test = filteredTags[int(0.80*df.shape[0]):df.shape[0],:]

In [ ]:
#Deleting tags and sequence columns as they are not required , only TitleBody and tags encoded are required
del x_train['Tags']
del x_train['Sequence']

del x_test['Tags']
del x_test['Sequence']

Now that we obtained our data in the required format we fit the data in bag of words and TfIdf (Term Frequency Inverse Document Frequency) representations:

In [ ]:
#Represntation of our title + body combo in bag of words form
Bagvector= CountVectorizer(min_df=0.00009, max_features=150000, tokenizer = lambda x: x.split(),  ngram_range=(1,2))

x_train_bow = Bagvector.fit_transform(x_train['TitleBody'])
x_test_bow = Bagvector.transform(x_test['TitleBody'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Represntation of our title + body combo in Term Frequency Inverse Document Frequency (TFIDF) form
tfidf_vector = TfidfVectorizer(min_df=0.00009, max_features=150000, smooth_idf=True, norm="l2", tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,4))

x_train_tf = tfidf_vector.fit_transform(x_train['TitleBody'])
x_test_tf = tfidf_vector.transform(x_test['TitleBody'])

In [ ]:
#Importing required packages for classifiers and accuracy calculation
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score

In [ ]:
print("Dimensions of train data X:",x_train_bow.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",x_test_bow.shape,"Y:",y_test.shape)

Dimensions of train data X: (48, 2827) Y : (48, 40)
Dimensions of test data X: (12, 2827) Y: (12, 40)


In [ ]:
import pickle

In [ ]:
# Saving the data, necesarry when data is large to recover the features when memory limits exceeds
with open('x_store.pkl','wb') as f:
    pickle.dump((x_train_bow,x_test_bow,y_train,y_test),f)

x_train_bow,x_test_bow,y_train,y_test = pickle.load(open('x_store.pkl','rb'))

In [ ]:
# There is a scipy bug for sparse matrices, so we have used this to handle that bug
x_train_bow.sort_indices()

In [ ]:
#function to print the respective values for trained classifier
def values(pred, true):

  print("Accuracy : ",metrics.accuracy_score(true,pred))
  print("Macro f1 score : ",metrics.f1_score(true, pred, average = 'macro'))
  print("Micro f1 score : ",metrics.f1_score(true, pred, average = 'micro'))
  print("Hamming loss : ",metrics.hamming_loss(true,pred))

Using one vs rest classifier with STOCHASTIC GRADIENT DESCENT with BAG OF WORDS features and LOG LOSS (LOGISTIC CLASSIFIER WITH SGD) :

In [ ]:
LogBOW = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1',n_jobs=-1), n_jobs=-1)
LogBOW.fit(x_train_bow.copy(), y_train)
predictions = LogBOW.predict(x_test_bow.copy())

values(predictions, y_test)

Accuracy :  0.3333333333333333
Macro f1 score :  0.041666666666666664
Micro f1 score :  0.14814814814814817
Hamming loss :  0.04791666666666667


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
predictions.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Using one vs rest classifier with STOCHASTIC GRADIENT DESCENT with TERM FREQUENCY features and LOGISTIC CLASSIFIER  :

In [ ]:
LogTF = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.001, penalty='l1',n_jobs=-1), n_jobs=-1)
LogTF.fit(x_train_tf, y_train)
predictions = LogTF.predict(x_test_tf)

values(predictions, y_test)

Using one vs rest classifier with GRADIENT DESCENT with BAG OF WORDS features and LOGISTIC CLASSIFIER:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogBOW = OneVsRestClassifier(LogisticRegression(penalty='l1',verbose=1,n_jobs=-1), n_jobs=-1)
LogBOW.fit(x_train_bow.copy(), y_train)
predictions = LogBOW.predict(x_test_bow.copy())

values(predictions, y_test)

Using one vs rest classifier with GRADIENT DESCENT with TERM FREQUENCY features and LOGISTIC CLASSIFIER:

In [ ]:
LogTF = OneVsRestClassifier(LogisticRegression(penalty='l1',verbose=1,n_jobs=-1), n_jobs=-1)
LogTF.fit(x_train_tf, y_train)
predictions = LogTF.predict(x_test_tf)

values(predictions, y_test)

Using one vs rest classifier with STCHASTIC GRADIENT DESCENT with BAG OF WORDS features and SVM CLASSIFIER:

In [ ]:
LogBOW = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.00001, penalty='l1',n_jobs=-1), n_jobs=-1)
LogBOW.fit(x_train_bow.copy(), y_train)
predictions = LogBOW.predict(x_test_bow.copy())

values(predictions, y_test)


Using one vs rest classifier with STOCHASTIC GRADIENT DESCENT with TERM FREQUENCY features and SVM CLASSIFIER:

In [ ]:
SVMtf = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.001, penalty='l1',n_jobs=-1), n_jobs=-1)
SVMtf.fit(x_train_tf.copy(), y_train)
predictions = SVMtf.predict(x_test_tf.copy())

print_performance(predictions, y_test)

CODE for HYPER PARAMETER TUNING of various parameters:

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# grid containing various alpha (parameter) values
param_grid = {"estimator__C": [0.0001,0.005,0.001, 0.01, 0.1, 1, 10, 100] }

In [ ]:
# We changed the code to find alpha values for various types of classifiers we used in the project 
lgr = OneVsRestClassifier(LogisticRegression(penalty='l1',n_jobs=-1,),n_jobs=-1)
clf = GridSearchCV(lgr, param_grid,cv=3,n_jobs=-1)
clf.fit(x_train_bow, y_train)

In [ ]:
print(clf.best_params_)

In [ ]:
#Testing the predicted values for the obtained parameter
predictions = clf.predict(x_test_bow)
values(predictions, y_test)